<a href="https://colab.research.google.com/github/aruntakhur/LLMs/blob/main/GPT_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-18 08:37:43--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-04-18 08:37:43 (23.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
import torch
with open("input.txt","r",encoding = "utf-8") as f:
  text = f.read()
f.close()
print(text)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
device

'cuda'

In [7]:
print("length of the text ", len(text))

length of the text  1115394


In [8]:
print("first 1000 characters are ",text[:1000])

first 1000 characters are  First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, n

In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars), vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [10]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: "".join([itos[i] for i in l])

e_txt = encode(text)



In [11]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)

In [12]:
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [13]:
n = int(0.9*len(data))
train_data = data[:n]
val_data  = data[n:]
print(len(train_data), len(val_data))

1003854 111540


In [14]:
torch.randint(len(train_data)-8,(4,))

tensor([754611, 264867, 337346, 662238])

In [15]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
print("",x,"\n",y)
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"Whent the input is {context}; the target is {target}")


 tensor([18, 47, 56, 57, 58,  1, 15, 47]) 
 tensor([47, 56, 57, 58,  1, 15, 47, 58])
Whent the input is tensor([18]); the target is 47
Whent the input is tensor([18, 47]); the target is 56
Whent the input is tensor([18, 47, 56]); the target is 57
Whent the input is tensor([18, 47, 56, 57]); the target is 58
Whent the input is tensor([18, 47, 56, 57, 58]); the target is 1
Whent the input is tensor([18, 47, 56, 57, 58,  1]); the target is 15
Whent the input is tensor([18, 47, 56, 57, 58,  1, 15]); the target is 47
Whent the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]); the target is 58


In [16]:
torch.manual_seed(1337)
block_size = 8
batch_size = 4
def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x,y
xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---")

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(context,target)




inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')
---
tensor([24], device='cuda:0') tensor(43, device='cuda:0')
tensor([24, 43], device='cuda:0') tensor(58, device='cuda:0')
tensor([24, 43, 58], device='cuda:0') tensor(5, device='cuda:0')
tensor([24, 43, 58,  5], device='cuda:0') tensor(57, device='cuda:0')
tensor([24, 43, 58,  5, 57], device='cuda:0') tensor(1, device='cuda:0')
tensor([24, 43, 58,  5, 57,  1], device='cuda:0') tensor(46, device='cuda:0')
tensor([24, 43, 58,  5, 57,  1, 46], device='cuda:0') tensor(43, device='cuda:0')
tensor([24, 43, 58,  5, 57,  1, 46, 43], device='cuda:0') tensor(39, device='cuda

In [17]:
torch.manual_seed(1337)
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embedding_lookup_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self,idx,targets=None):
    logits = self.embedding_lookup_table(idx)
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim = -1)
      idx_next = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx, idx_next), dim = -1)
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)
# logits, loss = m(xb,yb)
# print(xb.shape)
# xb_new = m.generate(xb,5)
# print(xb_new)

start = torch.zeros((1,1), dtype = torch.long, device = device)
para = m.generate(start,100)
print(decode(para[0].tolist()))




pYCXxfRkRZd
wc'wfNfT;OLlTEeC K
jxqPToTb?bXAUG:C-SGJO-33SM:C?YI3a
hs:LVXJFhXeNuwqhObxZ.tSVrddXlaSZaNe


In [18]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [19]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
    model.train()
  return out


In [20]:
batch_size = 32
max_iters = 3000
eval_interval = 300
eval_iters = 200
for iter in range(max_iters):
  if iter % eval_interval == 0:
    loss = estimate_loss()
    print(f"iter: {iter}, train loss: {loss['train']:.4f}, val loss: {loss['val']:.4f}")
  xb, yb = get_batch('train')
  logits, loss = m(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
print(loss)

iter: 0, train loss: 4.7305, val loss: 4.7241
iter: 300, train loss: 4.3818, val loss: 4.3896
iter: 600, train loss: 4.0801, val loss: 4.0784
iter: 900, train loss: 3.8066, val loss: 3.8117
iter: 1200, train loss: 3.5844, val loss: 3.5850
iter: 1500, train loss: 3.3757, val loss: 3.3829
iter: 1800, train loss: 3.2182, val loss: 3.2218
iter: 2100, train loss: 3.0817, val loss: 3.0810
iter: 2400, train loss: 2.9663, val loss: 2.9739
iter: 2700, train loss: 2.8809, val loss: 2.8800
tensor(2.7940, device='cuda:0', grad_fn=<NllLossBackward0>)


In [21]:
print(decode(m.generate(torch.zeros((1,1),dtype = torch.long, device = device),500)[0].tolist()))


Wawicrd RWI,
DERacomzoroup
Yow&$FMOUf isth bHEv!$Whedillxcaeg ireeYERngmin latiHNGAdrov ts, anenWk p.
GRWilyWjbo!
el.lind me u.
-huD3SPy wiry:CUEOKMORT'X3Qw y. w'sBHUSInormopeYelgCIEJMk:
Gll, d motSPkllo W-woo whrVCeiib3s wor m dE$HZAETENGShireAs p-LK3:Cl-xTre

ALkOMmnterupt f s z; iris!
m:CENGjey aleUE$ERUNMadPrD?d KISo myaHKINLIk!
Ktiyb&y,:
SadaplWPT:VE:zLUYBinin cNuk?ayeaney Iry tsmI&fy VEc!3MykenEETonoke mary.
YoXy'WWh IUSPm sora anghse.-w?nomabyFq-wneY.
SanRY; stx;KIOUNGoth qHegibodERTI'duc


In [22]:
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a,1,keepdim = True)
b = torch.randint(0,10,(3,2)).float()
print("a",a,"b",b)
c = a@b
print(c)

a tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) b tensor([[1., 8.],
        [6., 4.],
        [5., 4.]])
tensor([[1.0000, 8.0000],
        [3.5000, 6.0000],
        [4.0000, 5.3333]])


In [23]:
xx = torch.arange(1,13)
xx = xx.view(3,2,2).float()
xbow = torch.zeros(xx.shape).float()
print(xx)
for t in range(2):
  xprev = xx[:,:t+1,:]
  xbow[:,t] = torch.mean(xprev,1)
xbow

tensor([[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[ 9., 10.],
         [11., 12.]]])


tensor([[[ 1.,  2.],
         [ 2.,  3.]],

        [[ 5.,  6.],
         [ 6.,  7.]],

        [[ 9., 10.],
         [10., 11.]]])

In [24]:
B, T, C = 3, 2, 2
a = torch.tril(torch.ones(T,T)).float()
a = a/torch.sum(a,1,keepdim = True)
a@xx

tensor([[[ 1.,  2.],
         [ 2.,  3.]],

        [[ 5.,  6.],
         [ 6.,  7.]],

        [[ 9., 10.],
         [10., 11.]]])

In [25]:
#version 3
a = torch.tril(torch.ones(T,T)).float()
wei = torch.zeros(T,T)
wei = wei.masked_fill(a==0,float('-inf'))
wei = F.softmax(wei,1)
wei@xx

tensor([[[ 1.,  2.],
         [ 2.,  3.]],

        [[ 5.,  6.],
         [ 6.,  7.]],

        [[ 9., 10.],
         [10., 11.]]])

In [26]:
k = torch.randn(B,T,C)
k@k.transpose(-2,1)

tensor([[[ 1.1553,  0.1445],
         [ 0.2312,  1.2090]],

        [[-0.0070, -0.4157],
         [ 0.3919, -0.3715]],

        [[-0.4289, -0.3727],
         [ 0.3823,  0.2862]]])

In [27]:
k.transpose(-3,-1)

tensor([[[-0.0722, -0.6061,  0.7754],
         [ 1.3565, -0.5941, -1.0280]],

        [[ 0.8479,  0.6301,  1.0021],
         [ 0.2426, -0.0536, -1.1473]]])

In [28]:
embedding_lookup_table = nn.Embedding(vocab_size, vocab_size)

In [29]:
xb

tensor([[56, 43,  1, 39, 57,  1, 43, 39],
        [53,  1, 57, 43, 43,  6,  1, 52],
        [51, 39, 49, 43, 57,  1, 59, 57],
        [58, 46, 43,  0, 56, 43, 57, 58],
        [58,  1, 58, 46, 43, 57, 43,  1],
        [43, 42, 47, 41, 39, 51, 43, 52],
        [42,  1, 57, 39, 63,  6,  1, 50],
        [43,  1, 42, 39, 52, 45, 43, 56],
        [42, 53, 11,  1, 39, 52, 42,  1],
        [45, 50, 39, 57, 57, 11,  0, 21],
        [43, 39, 58, 46,  5, 57,  1, 40],
        [43,  1, 51, 43,  6,  1, 58, 46],
        [42,  0, 58, 53,  1, 58, 46, 43],
        [59,  1, 46, 39, 60, 43,  1, 51],
        [ 1, 13, 33, 25, 17, 30, 24, 17],
        [56, 43,  1, 56, 43, 58, 59, 56],
        [43,  5, 43, 56,  1, 57, 54, 43],
        [47, 52, 45,  1, 55, 59, 43, 43],
        [ 1, 51, 43,  1, 58, 46, 39, 58],
        [47, 45, 46, 58,  0, 21,  1, 58],
        [43, 39, 42, 50, 63,  1, 46, 39],
        [25, 39, 56, 41, 47, 59, 57, 10],
        [ 0, 18, 53, 56,  1, 58, 39, 49],
        [39, 52, 49,  1, 58, 46, 4

In [30]:
#Attention
torch.manual_seed(1337)
embedding_lookup_table = nn.Embedding(vocab_size, vocab_size)
embedding_lookup_table.to(device)
data = embedding_lookup_table(xb)
B, T, C = data.shape
head_size = 16
query = nn.Linear(C, head_size, bias = False).to(device)
key = nn.Linear(C, head_size, bias = False).to(device)
value = nn.Linear(C, head_size, bias = False).to(device)
q = query(data)
k = key(data)
wei = q@k.transpose(-2,-1)

In [31]:
v = value(data)

In [32]:
tril = torch.tril(torch.ones(T,T)).to(device)
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,-1)
out = wei@v

In [33]:
out.shape

torch.Size([32, 8, 16])

In [34]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.8259e-02, 9.7174e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.0921e-02, 9.5321e-01, 5.8743e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [5.9018e-02, 1.7285e-02, 8.6363e-01, 6.0065e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [6.0881e-02, 8.1439e-01, 3.8450e-02, 6.8397e-02, 1.7881e-02, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [3.7669e-02, 8.7746e-01, 5.4075e-03, 5.4218e-02, 1.9843e-02, 5.4075e-03,
         0.0000e+00, 0.0000e+00],
        [1.4178e-02, 4.8753e-01, 6.9261e-04, 4.2647e-03, 5.1200e-03, 6.9261e-04,
         4.8753e-01, 0.0000e+00],
        [7.5861e-03, 2.2218e-03, 1.1101e-01, 7.7206e-03, 7.5051e-01, 1.1101e-01,
         2.2218e-03, 7.7206e-03]], device='cuda:0', grad_fn=<SelectBackward0>)

In [35]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [36]:
print(k.var(),q.var(),wei.var())

tensor(1.0075) tensor(0.9928) tensor(1.0165)


In [37]:
n_embd = 32
class Head(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.query = nn.Linear(n_embd, head_size, bias = False)
    self.key = nn.Linear(n_embd, head_size, bias = False)
    self.value = nn.Linear(n_embd, head_size, bias = False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)
  def forward(self, x):
    B, T, C = x.shape
    k = self.key(x)
    q = self.query(x)
    wei = q@k.transpose(-2,-1)*C**-0.5
    wei = wei.masked_fill(self.tril[:T,:T]==0, float('-inf'))
    wei = F.softmax(wei, -1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei@v
    return out


In [38]:
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding = nn.Embedding(block_size, n_embd)
    self.sa_head = Head(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets = None):
    B, T = idx.shape
    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.sa_head(x)
    logits = self.lm_head(x)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:, -block_size:]
      logits, _ = self(idx_cond)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim = -1)
      new_tok = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx, new_tok), dim = -1)
    return idx



In [39]:
batch_size = 32
max_iters = 5000
eval_interval = 300
eval_iters = 200
learning_rate = 1e-3
dropout = 0.0

model = BigramLanguageModel()
m = model.to(device)


optimizer = torch.optim.AdamW(m.parameters(), lr = learning_rate)

for iter in range(max_iters):
  if iter%eval_interval == 0:
    losses = estimate_loss()
    print(f"iter: {iter}, train_loss: {losses['train']:.4f}, val_loss: {losses['val']:.4f}")
  xb, yb = get_batch('train').to(device)
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()


iter: 0, train_loss: 4.2881, val_loss: 4.2856
iter: 300, train_loss: 2.9430, val_loss: 2.9614


KeyboardInterrupt: 

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

In [ ]:
decode(m.generate(torch.zeros((1,1),dtype = torch.long, device = device),500)[0].tolist())

In [ ]:
n_embd = 32
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList(Head(head_size) for _ in range(num_heads))
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([head(x) for head in self.heads], dim = -1)
    out = self.dropout(self.proj(out))
    return out

In [ ]:
class FeedForward(nn.Module):

  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(n_embd, n_embd),
                             nn.ReLU(),
                             )
  def forward(self, x):
    return self.net(x)




In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_heads = MultiHeadAttention(4, n_embd//4 )
    self.lm_head = nn.Linear(n_embd, vocab_size)
    self.ffwd = FeedForward()

  def forward(self, x, targets = None):
    B, T = x.shape
    tok_emb = self.token_embedding_table(x)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.sa_heads(x)
    x = self.ffwd(x)
    logits = self.lm_head(x)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self, x, max_new_tokens):
    for _ in range(max_new_tokens):
      x_cond = x[:,-block_size:]
      print(x_cond)
      logits, loss = self(x_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim = -1)
      nxt_token = torch.multinomial(probs, num_samples = 1)
      x = torch.cat((x, nxt_token), dim = -1)
    return x


In [ ]:
batch_size = 32
max_iters = 5000
eval_interval = 300
eval_iters = 200
learning_rate = 1e-3
dropout = 0.0

model = BigramLanguageModel()
m = model.to(device)


optimizer = torch.optim.AdamW(m.parameters(), lr = learning_rate)

for iter in range(10):
  if iter%eval_interval == 0 or iter == max_iters-1:
    losses = estimate_loss()
    print(f"iter: {iter}, train_loss: {losses['train']:.4f}, val_loss: {losses['val']:.4f}")
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()


In [ ]:
decode(m.generate(torch.zeros((1,1),dtype = torch.long, device = device),500)[0].tolist())

In [ ]:
class Block(nn.Module):
  def __init__(self, n_embd, num_head):
    super().__init__()
    self.heads = MultiHeadAttention(num_head, n_embd//num_head)
    self.ffwd = FeedForward()

  def forward(self, x):
    x = self.heads(x)
    x = self.ffwd(x)
    return x

In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self, n_embd, num_head):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(
        Block(n_embd, num_head),
        Block(n_embd, num_head),
        Block(n_embd, num_head),
        Block(n_embd, num_head),
    )
    self.lm_head = nn.Linear(n_embd, vocab_size)
    self.register_buffer('tril', torch.tril(torch.ones(T,T)))

  def forward(self, x, targets = None):
    B, T = x.shape
    tok_emb = self.token_embedding_table(x)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    logits = self.lm_head(x)
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits, _ = self(idx_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim = -1)
      new_tok = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx,new_tok), dim = -1)
    return idx

In [ ]:
num_head = 4
model = BigramLanguageModel(n_embd, num_head)
m = model.to(device)
optimizer = torch.optim.AdamW(m.parameters(),lr = learning_rate)

for iter in range(300):
  if iter % eval_interval == 0 or iter == max_iters-1:
    losses = estimate_loss()
    print(f"iter: {iter}, train error: {losses['train']:.4f}, val error: {losses['val']:.4f}")
  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()



In [ ]:
logits.shape

In [ ]:
decode(m.generate(torch.zeros((1,1), dtype = torch.long, device = device),500)[0].tolist())

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

# Final code

In [1]:
# Download the data from github
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-19 14:40:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-04-19 14:40:40 (29.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F

eval_interval = 300
eval_iters = 200
block_size = 8
batch_size = 4
split_size = 0.9
emb_size = 32
head_size = 8
num_heads = 4
num_layers = 6
dropout = 0.0
learning_rate = 1e-3
max_iters = 5000

In [32]:
# read the data from the file
device = 'cuda' if torch.cuda.is_available() else 'cpu'
text = ""
with open('input.txt','r',encoding = 'utf-8') as f:
  text = f.read()
print(len(text))

1115394


In [33]:
# Total vocabulary in the text file given character as token
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [34]:
# encoder and decoder for the text
ctoi = {ch:i for i,ch in enumerate(vocab)}
itoc = {i:ch for i,ch in enumerate(vocab)}
encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: [itoc[i] for i in l]
decode(encode("hi how are you"))

['h', 'i', ' ', 'h', 'o', 'w', ' ', 'a', 'r', 'e', ' ', 'y', 'o', 'u']

In [35]:
# encoding text for training the GPT and forming the training and validation data set
data = torch.tensor(encode(text))
l = int(split_size * len(data))
train_data = data[:l]
val_data = data[l:]
print(train_data[:100])


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [42]:
# defining a finction "get_batch" to create the batch of "batch_size" each of size equals to "block_size"
def get_batch(data):
  split = train_data if data == 'train' else val_data
  idx = torch.randint(0,len(split)-block_size,(batch_size,))
  xb = torch.stack([split[x:x+block_size] for x in idx ])
  yb = torch.stack([split[x+1:x+block_size+1] for x in idx])
  xb = xb.to(device)
  yb = yb.to(device)
  return xb, yb

In [43]:
# estimate the loss on both training and validation dataset
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for i in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[i] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out







In [44]:
class Head(nn.Module):
  """ one head of self-attention """

  def __init__(self, head_size):
    super().__init__()
    self.query = nn.Linear(emb_size, head_size, bias = False)
    self.key   = nn.Linear(emb_size, head_size, bias = False)
    self.value = nn.Linear(emb_size, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    wei = q @ k.transpose(-2, -1) * C **-0.5
    wei = wei.masked_fill(self.tril[:T,:T] == 0,float('-inf'))
    wei = F.softmax(wei, dim = -1)
    wei = self.dropout(wei)
    out = wei @ v
    return out

class MultiAttentionHead(nn.Module):
  """ multiple heads in parallel to compute self attention"""
  def __init__(self, head_size, num_heads):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(emb_size, emb_size)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([head(x) for head in self.heads], dim = -1)
    out = self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  """ simple neural network of two layers"""
  def __init__(self):
    super().__init__()

    self.net = nn.Sequential(
        nn.Linear(emb_size, 4*emb_size),
        nn.ReLU(),
        nn.Linear(4*emb_size, emb_size),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self, emb_size, n_heads):
    super().__init__()
    self.sa = MultiAttentionHead(emb_size//n_heads, n_heads)
    self.ffwd = FeedForward()
    self.lln1 = nn.LayerNorm(emb_size)
    self.lln2 = nn.LayerNorm(emb_size)

  def forward(self, x):
        #x = x + self.sa(self.ln1(x))
        #x = x + self.ffwd(self.ln2(x))
        x = x + self.lln1(self.sa(x))
        x = x + self.lln2(self.ffwd(x))
        return x

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, emb_size)
    self.pos_embedding_table = nn.Embedding(block_size, emb_size)
    self.blocks = nn.Sequential(*[Block(emb_size, num_heads) for _ in range(num_layers)])
    self.ln_f = nn.LayerNorm(emb_size)
    self.lm_head = nn.Linear(emb_size, vocab_size)

  def forward(self, x, targets = None):
    B, T = x.shape
    tok_emb = self.token_embedding_table(x)
    pos_emb = self.pos_embedding_table(torch.arange(T, device = device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      idx_cond = idx[:,-block_size:]
      logits, loss = self(idx_cond)
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim = -1)
      idx_new = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx, idx_new), dim = -1)
    return idx

In [45]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.080193 M parameters


In [46]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.3566, val loss 4.3379
step 300: train loss 2.7749, val loss 2.7927
step 600: train loss 2.6262, val loss 2.6244
step 900: train loss 2.5170, val loss 2.5775
step 1200: train loss 2.4726, val loss 2.4998
step 1500: train loss 2.4366, val loss 2.4347
step 1800: train loss 2.3821, val loss 2.4224
step 2100: train loss 2.3604, val loss 2.3843
step 2400: train loss 2.3401, val loss 2.4172
step 2700: train loss 2.3156, val loss 2.3243
step 3000: train loss 2.3024, val loss 2.3242
step 3300: train loss 2.3210, val loss 2.3088
step 3600: train loss 2.2855, val loss 2.3002
step 3900: train loss 2.2629, val loss 2.3046
step 4200: train loss 2.2672, val loss 2.2584
step 4500: train loss 2.2429, val loss 2.2457
step 4800: train loss 2.2326, val loss 2.2436
step 4999: train loss 2.2266, val loss 2.2673


In [47]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

['\n', 'M', 'R', 'E', 'N', 'G', '\n', 'K', 'I', 'N', 'G', '\n', 'H', 'a', 'n', 'd', '.', '\n', '\n', 'H', 'e', 'L', 'U', ':', '\n', 'T', 'o', ' ', 't', 'a', 'y', ' ', 'I', ' ', ',', '\n', 'M', 'y', 's', 't', 'e', 'r', 'd', ',', '\n', 'y', 'o', 'h', ' ', 't', 'o', ' ', 'b', 'u', 'n', ' ', 'a', 'n', 'n', 'd', ' ', 'p', 'e', 'l', 'l', ' ', 'm', 'o', 'u', 'l', 'd', '!', '\n', '\n', 'L', 'E', 'R', 'G', '\n', 'T', 'E', ' ', 'O', ':', '\n', 'T', 'a', 'r', 'm', 'o', 'u', 's', ',', ',', ' ', 'E', 't', ' ', 'm', 'e', ' ', 'b', 'e', ',', ' ', 'i', 'n', 'g', 'e', 't', ' ', 'd', 'i', 'n', 'e', 's', 'a', '!', ' ', 'h', 'e', 'n', ' ', 'h', 'e', 'r', 's', 't', 'o', 'm', 'e', ' ', 't', 'h', 'a', 't', ',', ' ', 't', 'h', 'e', 'e', ' ', 't', 'h', 'i', 'g', 'd', 'm', 'e', ' ', 'p', 'e', 'v', 'e', 'r', 'a', 'm', '-', 'e', ' ', 'a', 'n', 'd', ' ', 't', 'o', ' ', 'c', 'a', 'a', 't', 'h', 'f', ',', ' ', 't', 'h', 'e', 'r', ' ', 'h', 'o', 'o', ' ', 't', 'o', ' ', 'C', 'a', 'n', ' ', 'b', 'r', 'o', 't', ' ', 'w

In [49]:
"".join(decode(m.generate(torch.zeros((1,1), dtype = torch.long, device = device),500)[0].tolist()))

"\n\nThoust woughter hird wer arll a frarcune bjeesth frork, brads helle! and, motergeard I tas ceees as le opiteldos ineblany houre diiks nisom?\n\nKK: the lod of muse; a sittell lrice the gard, thag a thawe the yeell thouch afor here ithane a then that t goltert be toow. mu,\nY lamcur:\nWuly bestro dolwk cesive terlust;\nAt me tur bapt Ooncbe poit so-\nTe heave in pince brift too\n\nMuch y mu\nir: on ma she's haverst a not let torke wrilith un with lif Goon my I what yer shou hiem: ous you yoou ir;s fes to"